In [2]:
import seaborn
import ranking as r
import validate_model as vm
import create_model as cm
%matplotlib inline 
%run first_model.py
%load_ext autoreload
%autoreload 2

[INFO] This trial license of GraphLab Create is assigned to lax.cary@gmail.com and will expire on October 09, 2015. Please contact trial@dato.com for licensing options or to request a free non-commercial license for personal or academic use.

[INFO] Start server at: ipc:///tmp/graphlab_server-1864 - Server binary: /home/ubuntu/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1442329878.log
[INFO] GraphLab Server Version: 1.5.2


PROGRESS: Recsys training: model = factorization_recommender
PROGRESS: Preparing data set.
PROGRESS:     Data has 319332 observations with 30287 users and 109 items.
PROGRESS:     Data prepared in: 0.138466s
PROGRESS: Training factorization_recommender for recommendations.
PROGRESS: +--------------------------------+--------------------------------------------------+----------+
PROGRESS: | Parameter                      | Description                                      | Value    |
PROGRESS: +--------------------------------+--------------------------------------------------+----------+
PROGRESS: | num_factors                    | Factor Dimension                                 | 1        |
PROGRESS: | regularization                 | L2 Regularization on Factors                     | 0        |
PROGRESS: | solver                         | Solver used for training                         | sgd      |
PROGRESS: | max_iterations                 | Maximum Number of Iterations           

In [3]:
athlete_ratings, segment_ratings, models = cm.df_to_latent_features(training_df)

PROGRESS: Recsys training: model = factorization_recommender
PROGRESS: Preparing data set.
PROGRESS:     Data has 319332 observations with 30287 users and 109 items.
PROGRESS:     Data prepared in: 0.126067s
PROGRESS: Training factorization_recommender for recommendations.
PROGRESS: +--------------------------------+--------------------------------------------------+----------+
PROGRESS: | Parameter                      | Description                                      | Value    |
PROGRESS: +--------------------------------+--------------------------------------------------+----------+
PROGRESS: | num_factors                    | Factor Dimension                                 | 1        |
PROGRESS: | regularization                 | L2 Regularization on Factors                     | 0        |
PROGRESS: | solver                         | Solver used for training                         | sgd      |
PROGRESS: | max_iterations                 | Maximum Number of Iterations           

In [4]:
vm.testing_rmse(models, training_df)

{'downhill': 0.48497524664584696,
 'total': 0.3780499715071135,
 'uphill': 0.23350456828295837}

In [ ]:
vm.evaluate_latent_feature_correlations(training_df, segment_ratings)

In [ ]:
vm.plot_ratings(athlete_ratings)

In [ ]:
vm.plot_ratings(segment_ratings)

In [ ]:
leaderboards = r.get_all_leaderboards(athlete_ratings, file_path='../app/app_data/leaderboards.pkl')
leaderboards

In [5]:
df.query("seg_average_grade < 0").std()

average_watts         2.088545e+05
distance              1.895026e+03
elapsed_time          1.241465e+06
moving_time           5.383627e+02
athlete_id            1.618634e+06
segment_id            2.562684e+06
activity_id           1.117760e+08
seg_average_grade     2.143119e+00
seg_distance          1.863643e+03
seg_elevation_low     1.858260e+02
seg_elevation_high    1.913118e+02
seg_maximum_grade     3.909759e+01
tracks_cadence        4.430779e-01
tracks_heartrate      4.870112e-01
dist_diff             1.534885e+02
average_speed         1.324252e+00
dtype: float64

In [ ]:
df.groupby(['athlete_id', 'segment_id']).average_speed.mean().std()

In [ ]:
downhill_test_df = testing_df.query('seg_average_grade < 0')

In [ ]:
downhill_test_df.shape